In [89]:
import random

class JogoDeForca:
    def __init__(self):
        import requests
        url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
        r = requests.get(url, allow_redirects=True)
        if r.status_code==200:
            self.content = str(r.content.decode()).lower().split('\n')[:12400]
            # self.content = ['Sammy', 'Ashley', 'Jo', 'Olly', 'Jackie', 'Charlie']
        else:
            print("Erro: ", r.status_code)
    
    def novo_jogo(self, vidas=5):
        self.vidas = vidas
        self.palavra = random.choice(self.content)
        self.finalizado = False
        return len(self.palavra)

    def tentar_letra(self, letra):
        if self.vidas > 0:
            if letra in self.palavra:
                return [idx for idx in range(len(self.palavra)) if self.palavra[idx]==letra]
            else:
                self.vidas -= 1
                if self.vidas == 0:
                    # print("Fim de jogo!")
                    return False
                else:
                    return []
        else:
            # self.finalizado = True
            return False
        
    def tentar_palavra(self, palavra):
        if self.vidas > 0:
            if self.palavra == palavra:
                # print ("Ganhou!")
                return True
            else:
                self.vidas = 0
                # print("Fim de jogo!")
                return False
        else:
            self.finalizado = True

In [90]:
from sklearn.tree import DecisionTreeClassifier
# import sklearn.tree

class Jogador:
    def __init__(self, n_letras, jogo):
        self.n_letras = n_letras
        self.jogo = jogo
        self.letras_descartadas = ''
        self.letras_na_palavra = ''
        self.possivel_palavra = ['' for _ in range(n_letras)]
        self.palavras_possiveis = list(filter(lambda x: (len(x) == self.n_letras) , self.jogo.content))
        # self.palavras_possiveis = ['Sammy', 'Ashley', 'Jo', 'Olly', 'Jackie', 'Charlie']
    
    def lista_palavras(self, resultado, l):
        lista_nova = []
        if len(resultado) == 0:
            for p in self.palavras_possiveis:
                if l not in p:
                    lista_nova.append(p)
        else:
            for p in self.palavras_possiveis:
                pin = True
                for i in resultado:
                    if p[i] != l:
                        pin = False
                        break
                if pin:
                    lista_nova.append(p)
        self.palavras_possiveis = lista_nova
        
        return len(self.palavras_possiveis)
    
    def arvore_escolhas(self, df):

        tree = DecisionTreeClassifier(criterion='entropy')
        filtro = df['palavra'].isin(self.palavras_possiveis)
        df = df[filtro]
        letras = df.drop(['palavra'], axis=1)
        palavras = df['palavra']


        # Agora, vamos usar o método .fit() para ajustar os parâmetros da árvore:
        tree.fit(letras, palavras)

        w = tree.feature_importances_
        ranking = [x for _, x in sorted(zip(w,letras))][::-1]

        i = 0
        letra = ranking[i]
        while letra in self.letras_descartadas :
            i += 1
            letra = ranking[i]

        self.letras_descartadas += letra
        return letra
    
    def escolhe_letra(self):
        texto = ''.join(self.palavras_possiveis)
        dicio = {}
        for c in set(texto):
            # c.append(  {'prob':texto.count(c)/len(texto), 'valor':c} )
            dicio[c] = texto.count(c)/len(texto)

        sortedDict = sorted(dicio.items(), key=lambda x:x[1])
        i = len(sortedDict) -1
        letra = sortedDict[i][0]
        while letra in self.letras_descartadas and i >0 :
            i -= 1
            letra = sortedDict[i][0]

        self.letras_descartadas += letra
        
        return letra
    

        

In [91]:
import pandas as pd
# df = pd.read_csv('frequencia1.csv')
# df.head()

In [92]:
# filtro = []
# for i in df['palavra']:
#     if i in ['abacate', 'pedro']:
#         filtro.append(True)
#     else:
#         filtro.append(False)
# filtro = df['palavra'].isin(['abacate', 'pedro'])
# df = df[filtro]
# df.head()

In [ ]:
# from JogoDeForca import JogoDeForca
# from Jogador import Jogador
import random
import pandas as pd

df = pd.read_csv('frequencia1.csv')

vitorias = 0
chute = 0
chute_bom = 0
jogo = JogoDeForca()
for _ in range(100):
    n_letras = jogo.novo_jogo()
    # print(jogo.palavra)
    jogador = Jogador(n_letras, jogo)
    while True:
        if jogo.vidas > 2:
            l = jogador.escolhe_letra()
        else:
            l = jogador.arvore_escolhas(df) 
        resultado = jogo.tentar_letra(l)
        # print(l)
        # print(jogo.vidas)
        if type(resultado) == bool:
            r = resultado
            break
        else:
            if len(resultado) != 0:
                jogador.letras_na_palavra += l
            # print(resultado)
            a = jogador.lista_palavras(resultado, l)
            # print(jogador.palavras_possiveis)
            # print(a)
            if jogo.vidas == 1:
                chute += 1
                # print(jogo.palavra)
                # print(jogador.letras_descartadas)
                # print(jogador.palavras_possiveis)
                r = jogo.tentar_palavra(random.choice(jogador.palavras_possiveis))
                if r: 
                    chute_bom += 1
                break
            if a == 1:
                r = jogo.tentar_palavra(jogador.palavras_possiveis[0])
                # print(jogo.vidas)
                break
    
    if r :
        vitorias +=1

print(vitorias/100)
print(chute)
print(chute_bom)

In [107]:
# from JogoDeForca import JogoDeForca
# from Jogador import Jogador
import random
import pandas as pd

df = pd.read_csv('frequencia1.csv')

def testa_jogada(n_iteracoes, q_prob):
    relatorio = {}
    relatorio['vitorias'] = 0
    relatorio['chute'] = 0
    relatorio['chute bom'] = 0
    jogo = JogoDeForca()
    for _ in range(n_iteracoes):
        n_letras = jogo.novo_jogo()
        jogador = Jogador(n_letras, jogo)
        while True:
            if jogo.vidas > q_prob:
                l = jogador.arvore_escolhas(df) 
            else:
                l = jogador.escolhe_letra()
            resultado = jogo.tentar_letra(l)
            if type(resultado) == bool:
                r = resultado
                break
            else:
                if len(resultado) != 0:
                    jogador.letras_na_palavra += l

                a = jogador.lista_palavras(resultado, l)
                if jogo.vidas == 1:
                    relatorio['chute'] += 1
                    r = jogo.tentar_palavra(random.choice(jogador.palavras_possiveis))
                    if r: 
                        relatorio['chute bom'] += 1
                    break
                if a == 1:
                    r = jogo.tentar_palavra(jogador.palavras_possiveis[0])
                    break
        
        if r :
            relatorio['vitorias'] +=1
    relatorio['vitorias'] /= n_iteracoes
    return relatorio

for i in range(5):
    print(i)
    relatorio = testa_jogada(100, i)
    print(relatorio)
# print(relatorio['vitorias'])
# print(relatorio['chute'])
# print(relatorio['chute bom'])

0
{'vitorias': 0.61, 'chute': 52, 'chute bom': 13}
1
{'vitorias': 0.68, 'chute': 57, 'chute bom': 25}
2
{'vitorias': 0.94, 'chute': 17, 'chute bom': 11}
3
{'vitorias': 0.94, 'chute': 12, 'chute bom': 6}
4
{'vitorias': 0.96, 'chute': 10, 'chute bom': 6}


0
{'vitorias': 0.98, 'chute': 7, 'chute bom': 3}
1
{'vitorias': 0.935, 'chute': 16, 'chute bom': 3}
2
{'vitorias': 0.94, 'chute': 18, 'chute bom': 6}
3
{'vitorias': 0.95, 'chute': 20, 'chute bom': 10}
4
{'vitorias': 0.9, 'chute': 35, 'chute bom': 15}